# Step 1: Convert xlsx metadata to csv format.

In [ ]:
%%bash
in2csv raw_data/anderson_annotated_data.xlsx > data/metadata.csv

# Step 2: Convert Nexus tree to Newick tree

I needed to also extract the accession numbers as the node names in the tree.
The script `nexus_to_newick.py` just converts the data.
The script `process_tree.py` parses the node names to split the accession numbers (the node name) and from the
other metadata, which is added as comments. I also had to manually parse the string to remove some weird backslashes,
single quotations, and square brackets added by biopython. The output tree has been tested on https://taxonium.org/


In [ ]:
%%bash
python scripts/nexus_to_newick.py raw_data/anderson_nexus_tree.txt data/raw_tree.nwk
python scripts/process_tree.py data/raw_tree.nwk data/tree.nwk

We now have a nice newick tree and it's associated metadata, and we could use any tree viewing tool to look at the data.
The following steps are optional and only needed to retreive the sequence data.

# Step 3: Download sequence data

The sequence data is stored at https://www.bv-brc.org/

In [ ]:
%%bash

# Extract accessions
csvcut -c Accession data/metadata.csv | tail -n +2 > data/accns.txt

# Download accessions from bv-brc
python scripts/download_bvbrc.py -i data/accns.txt -o data/downloads/

# Extract fasta from jsonl download
python scripts/extract_fasta.py -i data/downloads/raw_data.jsonl -o data/sequences.fasta

# Step 4: Align sequence data

Used A/New York City/2/1918(H1N1) from genbank as the alignment reference (accession OR133599).

In [ ]:
%%bash

mafft --thread 8 --auto --keeplength --addfragments data/sequences.fasta data/reference.fasta > data/sequences.aligned.fasta